# Parallel Computing made even simpler
We've already seen some examples of multithreaded and distributed programming, and it was already pretty simple. Just slap a macro in front of your `for` loop.

But the fun does not stop there. There are a couple of really nice packages that allow to define computations like folds (also known as Map-Reduce) and then to choose whether to run them single threaded, multithreaded or distributed (and even on the GPU).

You may have to run the next cell to install a couple packages, otherwise skip it.

In [ ]:
using Pkg
Pkg.add("Folds")
Pkg.add("Transducers")

In [2]:
using Folds
using Transducers
using Distributed

The `Distributed` package allows us to get some processes to run our code in parallel using the `addprocs` function. Let's get four of them.

In [3]:
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

As a simple example, we'll try to compute the number of palindromic primes under $10^8$.

Let's start by defining functions that return whether a given integer is a prime, or palindromic. Notice the `@everywhere` macro, which makes these functions known to all our processes.

In [4]:
@everywhere function isprime(n)
    k = 2
    while k^2 <= n
        if mod(n,k) == 0
            return false
        end
        k += 1
    end
    return true
end

In [5]:
@everywhere function ispalindromic(n)
    let xs = digits(n)
        xs == reverse(xs)
    end
end

In [23]:
N = 10^8

100000000

The next two lines show different ways of computing what we want. The first one uses `Folds.sum` with the functionalities from `Transducers`, and must be read from left to right (the `|>` is the pipe operator).

By default, `Folds` runs in multithreaded mode (which is pretty slow in this case because the notebook only has a single thread).

In [24]:
1:N |> Filter(isprime) |> Filter(ispalindromic) |> Map(n -> 1) |> Folds.sum

782

This next line uses the `foldxd` function from transducers, which does everything distributed.

In [25]:
foldxd(+, (1 for n in 1:N if isprime(n) && ispalindromic(n)))

782

For more complicated Map-Reduce operations, the `FLoops` package allows for specifying a `for` loop with (almost) arbitrary operations, and it is compatible with the various execution modes from `Folds`.